In [1]:
#importing libraries
import torch
from torch.autograd import Variable
from data import BaseTransform, VOC_CLASSES as labelmap
from ssd import build_ssd
import imageio
import cv2

In [2]:
#Implementing Detect function
def detect(frame, net, transform):
    height, width = frame.shape[:2]
    frame_t = transform(frame)[0]
    x = torch.from_numpy(frame_t).permute(2, 0, 1)
    x = Variable(x.unsqueeze(0))
    y = net(x)
    detections = y.data
    scale = torch.Tensor([width, height, width, height])
    # detections = [batch, number of classes, number of occurence, (score, x0, Y0, x1, y1)]
    for i in range(detections.size(1)):
        j = 0
        while detections[0, i, j, 0] >= 0.6:
            pt = (detections[0, i, j, 1:] * scale).numpy()
            cv2.rectangle(frame, (int(pt[0]), int(pt[1])), (int(pt[2]), int(pt[3])), (255, 0, 0), 2)
            cv2.putText(frame, labelmap[i - 1], (int(pt[0]), int(pt[1])), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            j += 1
    return frame

In [3]:
#building SSD Model
net = build_ssd('test')
net.load_state_dict(torch.load('ssd300_mAP_77.43_v2.pth', map_location = lambda storage, loc: storage))

D:\studies\Python\Computer Vision\[FreeTutorials.Us] Udemy - computer-vision-a-z\Computer Vision A-Z_code\Module 2 - Object Detection\Code for Windows\ssd.py:33: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)
D:\studies\Python\Computer Vision\[FreeTutorials.Us] Udemy - computer-vision-a-z\Computer Vision A-Z_code\Module 2 - Object Detection\Code for Windows\layers\modules\l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)


In [4]:
#Transformation
transform=BaseTransform(net.size, (104/256.0, 117/256.0, 123/256.0))

In [5]:
#loading the video
reader=imageio.get_reader('output.avi')
fps=reader.get_meta_data()['fps']
writer=imageio.get_writer('output_myself.mp4',fps=fps)
for i,frame in enumerate(reader):
    frame=detect(frame,net.eval(),transform)
    writer.append_data(frame)
    print(i)
writer.close()
      




D:\studies\Python\Computer Vision\[FreeTutorials.Us] Udemy - computer-vision-a-z\Computer Vision A-Z_code\Module 2 - Object Detection\Code for Windows\ssd.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  self.softmax(conf.view(-1, self.num_classes)),  # conf preds


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
